PyTorch 1일차 학습

In [2]:
import torch
import pandas as pd
import numpy as np

# --- 1. 가상의 데이터 생성 (csv 파일을 읽는 부분)
# 전세 사기 예측을 위한 간단한 가상 데이터를 만들어봅시다.
# feature: 전세가율(jeonse_ratio), 근저당비율(debt_ratio)
# label: 안전(0), 위험(1)
data = {
    'jeonse_ratio': [0.8, 0.9, 0.7, 0.95, 0.85],
    'debt_ratio': [0.1, 0.3, 0.0, 0.4, 0.2],
    'risk_label': [0, 1, 0, 1, 1]
}

# --- 2. Pandas DataFrame으로 데이터 불러오기
df = pd.DataFrame(data)
print("1. Pandas DataFrame:")
print(df)
print("-"*30)

# --- 3. NumPy 배열로 변환하기
# 모델 학습에는 순수한 숫자 데이터만 필요하므로 .values 속성을 사용
# 예측의 입력이 될 feature와 정답인 label을 분리
features_np = df[['jeonse_ratio', 'debt_ratio']].values
labels_np = df['risk_label'].values

print("2. NumPy Array (Features):")
print(features_np)
print(f"Type: {type(features_np)}")
print("-"*30)

# --- 4. PyTorch 텐서로 변환하기!
# torch.tensor() 함수를 사용하여 NumPy 배열을 PyTorch 텐서로 변환
features_tensor = torch.tensor(features_np, dtype=torch.float32)
labels_tensor = torch.tensor(labels_np, dtype=torch.float32)

print("3. PyTorch Tensor (Features):")
print(features_tensor)
print(f"Type: {type(features_tensor)}")
print(f"Shape: {features_tensor.shape}") # 모양 확인 (5개 데이터, 2개 feature)
print(f"Data Type: {features_tensor.dtype}") # 데이터 타입 확인
print("-" * 30)

# 레이블 텐서도 확인
print("4. PyTorch Tensor (Labels):")
print(labels_tensor)
print(f"Shape: {labels_tensor.shape}") # 모양 확인 (5개 데이터)

# 참고: 레이블 텐서의 차원을 모델 입력에 맞게 [5, 1] 형태로 변경해주면 더 좋습니다.
labels_tensor = labels_tensor.view(-1, 1)
print("\nReshaped Labels Tensor:")
print(labels_tensor)
print(f"Shape: {labels_tensor.shape}") # 모양 확인 ([5, 1])

1. Pandas DataFrame:
   jeonse_ratio  debt_ratio  risk_label
0          0.80         0.1           0
1          0.90         0.3           1
2          0.70         0.0           0
3          0.95         0.4           1
4          0.85         0.2           1
------------------------------
2. NumPy Array (Features):
[[0.8  0.1 ]
 [0.9  0.3 ]
 [0.7  0.  ]
 [0.95 0.4 ]
 [0.85 0.2 ]]
Type: <class 'numpy.ndarray'>
------------------------------
3. PyTorch Tensor (Features):
tensor([[0.8000, 0.1000],
        [0.9000, 0.3000],
        [0.7000, 0.0000],
        [0.9500, 0.4000],
        [0.8500, 0.2000]])
Type: <class 'torch.Tensor'>
Shape: torch.Size([5, 2])
Data Type: torch.float32
------------------------------
4. PyTorch Tensor (Labels):
tensor([0., 1., 0., 1., 1.])
Shape: torch.Size([5])

Reshaped Labels Tensor:
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [1.]])
Shape: torch.Size([5, 1])


2일차

In [3]:
import torch

# 1일차에 만든 텐서 (다시 생성)
# Shape: [5, 2] (5개 데이터, 2개 feature)
features_tensor = torch.tensor([
    [0.8, 0.1],
    [0.9, 0.3],
    [0.7, 0.0],
    [0.95, 0.4],
    [0.85, 0.2]
], dtype=torch.float32)

print("원본 텐서:")
print(features_tensor)

# 1-1. 수학 연산 (모든 '전세가율'에 0.1씩 더하기)
# (:, 0) -> 모든 행(:)의 0번째 열(전세가율)
features_tensor[:, 0] = features_tensor[:, 0] + 0.1
print("\n전세가율 + 0.1 연산:")
print(features_tensor)

# 1-2. 모양 변경 (.view() 사용)
# 현재 [5, 2] 모양 -> [1, 10] 모양으로 바꾸기 (데이터 10개가 쭉 이어진 형태)
# -1은 "나머지는 알아서 계산해"라는 뜻입니다.
# 5*2=10이므로, .view(1, 10) 또는 .view(-1) (1차원으로 펴기) 등과 같습니다.
reshaped_tensor = features_tensor.view(1, 10)
print(f"\n모양 변경 [5, 2] -> {reshaped_tensor.shape}:")
print(reshaped_tensor)

원본 텐서:
tensor([[0.8000, 0.1000],
        [0.9000, 0.3000],
        [0.7000, 0.0000],
        [0.9500, 0.4000],
        [0.8500, 0.2000]])

전세가율 + 0.1 연산:
tensor([[0.9000, 0.1000],
        [1.0000, 0.3000],
        [0.8000, 0.0000],
        [1.0500, 0.4000],
        [0.9500, 0.2000]])

모양 변경 [5, 2] -> torch.Size([1, 10]):
tensor([[0.9000, 0.1000, 1.0000, 0.3000, 0.8000, 0.0000, 1.0500, 0.4000, 0.9500,
         0.2000]])


In [4]:
# 2-1. 추적할 텐서 생성
# w와 b는 나중에 모델의 '가중치'와 '편향'이 될, 학습 대상 파라미터입니다.
# requires_grad=True로 설정하여 연산 과정을 추적하도록 합니다.
w = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(1.0, requires_grad=True)

# 가상의 입력 데이터 x
x = torch.tensor(3.0)

# 2-2. 연산 수행 (y = w * x + b 라는 간단한 모델)
# y는 w, b와 연관되어 있으므로, y도 자동으로 추적됩니다.
y = w * x + b
print(f"w, b, x로 계산된 y의 값: {y}") # y = 2.0 * 3.0 + 1.0 = 7.0

# 2-3. 기울기 계산 (가장 핵심적인 부분!)
# y에 대해 .backward()를 호출합니다.
# "y의 값(7.0)에 대해, w와 b가 얼마나 영향을 미쳤는지 계산해줘!"
y.backward()

# 2-4. 기울기 확인 (w.grad, b.grad)
# y = w*x + b 를 w로 미분하면 -> x (즉, 3.0)
# y = w*x + b 를 b로 미분하면 -> 1 (즉, 1.0)
print(f"\ny를 w로 미분한 값 (w의 기울기): {w.grad}")
print(f"y를 b로 미분한 값 (b의 기울기): {b.grad}")

# w.grad가 3.0이라는 의미: "w 값을 1만큼 증가시키면, y 값은 3만큼 증가합니다."
# b.grad가 1.0이라는 의미: "b 값을 1만큼 증가시키면, y 값은 1만큼 증가합니다."

w, b, x로 계산된 y의 값: 7.0

y를 w로 미분한 값 (w의 기울기): 3.0
y를 b로 미분한 값 (b의 기울기): 1.0


3일차